In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model

In [2]:
#load Dataset
df = pd.read_csv('/Users/User/Desktop/PUBG_match_data.csv')

In [3]:
df = df.drop(df[df['kills'] > 7].index)
df = df.drop(df[df['heals'] > 12].index)
df = df.drop(df[df['boosts'] > 7].index)
df = df.drop(df[df['walkDistance'] > 4396].index)
df = df.drop(df[df['swimDistance'] > 123].index)
df = df.drop(df[df['rideDistance'] > 6966].index)
df = df.drop(df[df['winPlacePerc'].isna()].index)
df['healsandboosts'] = df['heals'] + df['boosts']

In [4]:
def create_group_stats(df, col):

    group_stats = df.groupby('groupId')[col].agg(['max', 'min', 'mean']).reset_index()
    group_stats = group_stats.rename({'max': f'{col}_max', 'min': f'{col}_min', 'mean': f'{col}_mean'}, axis=1)
    df = pd.merge(df, group_stats, on='groupId')

    return df

In [5]:
col_list = ['kills', 'assists', 'healsandboosts', 'walkDistance', 'swimDistance', 'rideDistance','damageDealt', 'weaponsAcquired']
for col in col_list:
    
    df = create_group_stats(df, col)

In [6]:
#Train_test_split dataset
target = 'winPlacePerc'
drop_col = ['winPlacePerc','Id','matchType','heals','boosts','kills','damageDealt','maxPlace','matchDuration', 'assists', 'healsandboosts', 'walkDistance', 'swimDistance', 'rideDistance', 'weaponsAcquired','groupId','matchId']
y = df[target]
X = df.drop(columns = drop_col)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle = True)

In [7]:
#Standard_Scaler
scaler = StandardScaler().fit(train_X)
train_X = scaler.transform(train_X)

In [8]:
def get_model(input_size):
    input_X = Input((input_size, ))
    X = layers.Dense(32, activation = 'sigmoid')(input_X)
    X = layers.Dropout(0.3)(X)
    X = layers.Dense(32, activation = 'sigmoid')(X)
    output = layers.Dense(1, activation = 'sigmoid')(X)
    
    model = Model(inputs = input_X, outputs = output)
    
    return model

In [9]:
input_size = train_X.shape[1]

In [10]:
m1 = get_model(input_size)

In [11]:
m1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 37)]              0         
                                                                 
 dense (Dense)               (None, 32)                1216      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,305
Trainable params: 2,305
Non-trainable params: 0
_________________________________________________________________


In [12]:
opt = optimizers.Adam(learning_rate = 0.01, epsilon = 1e-8, amsgrad = False)
m1.compile(optimizer = opt, loss='mean_absolute_error', metrics=['mae'])

In [ ]:
history = m1.fit(train_X, train_y, batch_size = 2048, epochs = 50, validation_data = (test_X, test_y), verbose = 1)

Epoch 1/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0803 - mae: 0.0803 - val_loss: 0.3641 - val_mae: 0.3641
Epoch 2/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0690 - mae: 0.0690 - val_loss: 0.3362 - val_mae: 0.3362
Epoch 3/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0672 - mae: 0.0672 - val_loss: 0.2633 - val_mae: 0.2633
Epoch 4/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0662 - mae: 0.0662 - val_loss: 0.2417 - val_mae: 0.2417
Epoch 5/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0657 - mae: 0.0657 - val_loss: 0.2383 - val_mae: 0.2383
Epoch 6/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0653 - mae: 0.0653 - val_loss: 0.2394 - val_mae: 0.2394
Epoch 7/50
1651/1651 [==============================] - 2s 1ms/step - loss: 0.0651 - mae: 0.0651 - val_loss: 0.2469 - val_mae: 0.2469
Epoch 8/50
1651/1651 [==============================] - 2s 1ms

In [ ]:
plt.figure(figsize = (40,15))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()